```
docker run -it \
    --rm \
    --name elasticsearch \
    -m 4GB \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.17.6
```

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Question 2

In [3]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch("http://127.0.0.1:9200")

index_name = "faq-index"

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "question": {"type": "text"},
            "section": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

if not es_client.indices.exists(index=index_name):
    es_client.indices.create(index=index_name, body=index_settings)

from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

/Users/olgabraginskaya/.pyenv/versions/3.12.2/envs/llm-zoomcamp-2025/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 948/948 [00:01<00:00, 493.95it/s]


## Question 3

In [4]:
query = "How do execute a command on a Kubernetes pod?"

search_query = {
    "size": 1,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

response = es_client.search(index="faq-index", body=search_query)
top_hit = response["hits"]["hits"][0]
score = top_hit["_score"]
print("Top score:", round(score, 2))


Top score: 44.51


## Question 4

In [10]:
query = "How do copy a file to a Docker container?"

search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(index="faq-index", body=search_query)

for i, hit in enumerate(response["hits"]["hits"], 1):
    print(f"{i}. {hit['_source']['question']}")


1. How do I debug a docker container?
2. How do I copy files from my local machine to docker container?
3. How do I copy files from a different folder into docker container’s working directory?


## Question 5

In [11]:
query = "How do I execute a command in a running docker container?"

context_template = """
Q: {question}
A: {text}
""".strip()

context_parts = []
for hit in response["hits"]["hits"]:
    q = hit["_source"]["question"]
    a = hit["_source"]["text"]
    context_parts.append(context_template.format(question=q, text=a))

context = "\n\n".join(context_parts)

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

prompt = prompt_template.format(question=query, context=context)

print("Prompt length:", len(prompt))


Prompt length: 1462


## Question 6

In [12]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

token_count = len(encoding.encode(prompt))
print("Token count:", token_count)

Token count: 322
